# Init

In [1]:
%load_ext autoreload

In [2]:
from llmlib.app import app
context = app.app_context()
context.push()

## Imports

In [3]:
from develop.utils import formatters

## Inputs

In [4]:
query="What is the relationship between Zn2+ and glycolate?"

## Utils

In [5]:
import openai

def chatGPTQA(query: str):
    return openai.ChatCompletion.create(
        messages=[
            { "role": "user", "content": query }
        ],
        model="gpt-3.5-turbo",
        temperature=0
    )

## Initial check

Current response:

In [6]:
chatGPTQA(query)

<OpenAIObject chat.completion id=chatcmpl-8Dsd1HYGNEVkU7CY7IO36lAjJhkM3 at 0x7fe532386e70> JSON: {
  "id": "chatcmpl-8Dsd1HYGNEVkU7CY7IO36lAjJhkM3",
  "object": "chat.completion",
  "created": 1698318947,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The relationship between Zn2+ and glycolate is that Zn2+ can form a complex with glycolate. In this complex, the Zn2+ ion binds to the oxygen atoms in the glycolate molecule, forming coordination bonds. This interaction between Zn2+ and glycolate allows for the stabilization and transport of glycolate in biological systems."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 21,
    "completion_tokens": 77,
    "total_tokens": 98
  }
}

# Compose chain

## Init llm model

In [7]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, verbose=True)
llm

ChatOpenAI(cache=None, verbose=True, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='***OPENAI_API_KEY***', openai_api_base='', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None)

## Pepare prompts for core term extraction

In [8]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
QUERY_KEY = 'query'

core_terms_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            template="Always response with comma separated list of chemical or biological terms identified in prompt.",
            input_variables=[]
        )
    ),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=f"{{{QUERY_KEY}}}",
            input_variables=[QUERY_KEY],
        )
    )
])
core_terms_prompt_template

ChatPromptTemplate(input_variables=['query'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Always response with comma separated list of chemical or biological terms identified in prompt.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], output_parser=None, partial_variables={}, template='{query}', template_format='f-string', validate_template=True), additional_kwargs={})])

## Define embedding model

In [9]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base='', openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='***OPENAI_API_KEY***', openai_organization='', allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6, request_timeout=None, headers=None, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False)

## Define vectorstore

In [10]:
from langchain.vectorstores import Chroma

In [11]:
# Vectorstore
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_db_oai")
vectorstore

## Get graph reference

### Define neo4j connector

In [12]:
from llmlib.database import Neo4j

In [13]:
graph = Neo4j().graph()
graph.schema = ''  # Overwrite schema to avoid it to be loaded from db (lengthy process not ussed in here)

#### Init graph search API wrapper

In [14]:
from llmlib.utils.search.cypher_search_api_wrapper import CypherSearchAPIWrapper

In [15]:
search = CypherSearchAPIWrapper(
    graph=graph,
    verbose=True,
    return_intermediate_steps=True
)
search

CypherSearchAPIWrapper(graph=<llmlib.utils.lazy_neo4j_graph.LazyNeo4jGraph object at 0x7fe532106090>, related_nodes_query='\nUNWIND $terms AS term\nMATCH (s:Synonym {lowercase_name: toLower(term)})<-[:HAS_SYNONYM]-(n)\nWHERE n.eid IS NOT NULL AND NOT n.eid IN $exclude_eids\nWITH apoc.text.levenshteinSimilarity(term, n.name) as similarity, s, n, term\nORDER BY similarity DESC\nWHERE similarity > 0.5\nWITH DISTINCT term, collect({node: n, node_id: id(n), similarity: similarity})[0..$top_k_matches] as matches\nRETURN term, matches\n', related_nodes_query_params={'top_k_matches': 5, 'exclude_eids': ['NAD-P-OR-NOP', 'NADH-P-OR-NOP', 'Donor-H2', 'Acceptor', 'HYDROGEN-PEROXIDE', 'OXYGEN-MOLECULE', 'NAD', 'NADP', 'NADH', 'NADPH', 'WATER', 'CARBON-DIOXIDE', 'FAD', 'CO-A', 'UDP', 'AMMONIA', 'NA+', 'AMMONIUM', 'PROTON', 'CARBON-MONOXIDE', 'GTP', 'ADP', 'GDP', 'AMP', 'ATP', '3-5-ADP', 'PPI', 'Pi']}, relationships_query="\nUNWIND apoc.coll.combinations($term_match, 2) AS termPair\nUNWIND termPair[0]['matches'] as matchA\nUNWIND termPair[1]['matches'] as matchB\nWITH matchA, matchB, matchA['similarity'] * matchB['similarity'] AS similarity\nORDER BY similarity DESC\nWHERE similarity > 0.5\nMATCH (a), (b)\nWHERE id(a) = matchA['id'] AND id(b) = matchB['id']\nMATCH path=allShortestPaths((a)-[*1..]-(b))\nWHERE\n all(r in relationships(path) where type(r) in $include_rels)\n and all(n in nodes(path) where not n.eid in $exclude_eids)\nRETURN path LIMIT $top_k\n", relationships_query_params={'top_k': 3, 'include_rels': ['CONSUMED_BY', 'PRODUCES', 'CATALYZES', 'REGULATES', 'HAS_TXONOMY', 'IS', 'HAS_KO', 'ELEMENT_OF', 'ENCODES'], 'exclude_eids': ['NAD-P-OR-NOP', 'NADH-P-OR-NOP', 'Donor-H2', 'Acceptor', 'HYDROGEN-PEROXIDE', 'OXYGEN-MOLECULE', 'NAD', 'NADP', 'NADH', 'NADPH', 'WATER', 'CARBON-DIOXIDE', 'FAD', 'CO-A', 'UDP', 'AMMONIA', 'NA+', 'AMMONIUM', 'PROTON', 'CARBON-MONOXIDE', 'GTP', 'ADP', 'GDP', 'AMP', 'ATP', '3-5-ADP', 'PPI', 'Pi']})

## Compose QA chain with graph retriever

### Init graph retriever

In [16]:
from llmlib.utils.retrievers.graph_search_retriever import GraphSearchRetriever

In [17]:
retriever = GraphSearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm,
    graph_search=search,
    prompt=core_terms_prompt_template,
    verbose=True,
    return_intermediate_steps=True
)
retriever

GraphSearchRetriever(tags=None, metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7fe5321bfb10>, llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['query'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Always response with comma separated list of chemical or biological terms identified in prompt.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], output_parser=None, partial_variables={}, template='{query}', template_format='f-string', validate_template=True), additional_kwargs={})]), llm=ChatOpenAI(cache=None, verbose=True, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='***OPENAI_API_KEY***', openai_api_base='', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None), output_key='text', output_parser=CommaSeparatedListOutputParser(), return_final_only=True, llm_kwargs={}), graph_search=CypherSearchAPIWrapper(graph=<llmlib.utils.lazy_neo4j_graph.LazyNeo4jGraph object at 0x7fe532106090>, related_nodes_query='\nUNWIND $terms AS term\nMATCH (s:Synonym {lowercase_name: toLower(term)})<-[:HAS_SYNONYM]-(n)\nWHERE n.eid IS NOT NULL AND NOT n.eid IN $exclude_eids\nWITH apoc.text.levenshteinSimilarity(term, n.name) as similarity, s, n, term\nORDER BY similarity DESC\nWHERE similarity > 0.5\nWITH DISTINCT term, collect({node: n, node_id: id(n), similarity: similarity})[0..$top_k_matches] as matches\nRETURN term, matches\n', related_nodes_query_params={'top_k_matches': 5, 'exclude_eids': ['NAD-P-OR-NOP', 'NADH-P-OR-NOP', 'Donor-H2', 'Acceptor', 'HYDROGEN-PEROXIDE', 'OXYGEN-MOLECULE', 'NAD', 'NADP', 'NADH', 'NADPH', 'WATER', 'CARBON-DIOXIDE', 'FAD', 'CO-A', 'UDP', 'AMMONIA', 'NA+', 'AMMONIUM', 'PROTON', 'CARBON-MONOXIDE', 'GTP', 'ADP', 'GDP', 'AMP', 'ATP', '3-5-ADP', 'PPI', 'Pi']}, relationships_query="\nUNWIND apoc.coll.combinations($term_match, 2) AS termPair\nUNWIND termPair[0]['matches'] as matchA\nUNWIND termPair[1]['matches'] as matchB\nWITH matchA, matchB, matchA['similarity'] * matchB['similarity'] AS similarity\nORDER BY similarity DESC\nWHERE similarity > 0.5\nMATCH (a), (b)\nWHERE id(a) = matchA['id'] AND id(b) = matchB['id']\nMATCH path=allShortestPaths((a)-[*1..]-(b))\nWHERE\n all(r in relationships(path) where type(r) in $include_rels)\n and all(n in nodes(path) where not n.eid in $exclude_eids)\nRETURN path LIMIT $top_k\n", relationships_query_params={'top_k': 3, 'include_rels': ['CONSUMED_BY', 'PRODUCES', 'CATALYZES', 'REGULATES', 'HAS_TXONOMY', 'IS', 'HAS_KO', 'ELEMENT_OF', 'ENCODES'], 'exclude_eids': ['NAD-P-OR-NOP', 'NADH-P-OR-NOP', 'Donor-H2', 'Acceptor', 'HYDROGEN-PEROXIDE', 'OXYGEN-MOLECULE', 'NAD', 'NADP', 'NADH', 'NADPH', 'WATER', 'CARBON-DIOXIDE', 'FAD', 'CO-A', 'UDP', 'AMMONIA', 'NA+', 'AMMONIUM', 'PROTON', 'CARBON-MONOXIDE', 'GTP', 'ADP', 'GDP', 'AMP', 'ATP', '3-5-ADP', 'PPI', 'Pi']}), verbose=True)

### Define callback handler

In [18]:
from langchain.callbacks.stdout import StdOutCallbackHandler
handler = StdOutCallbackHandler()
handler

### Init QA chain

In [19]:
from langchain.chains import RetrievalQA

In [20]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True,
    callbacks=[handler]
)
qa

RetrievalQA(memory=None, callbacks=[<langchain.callbacks.stdout.StdOutCallbackHandler object at 0x7fe530afb0d0>], callback_manager=None, verbose=True, tags=None, metadata=None, combine_documents_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]), llm=ChatOpenAI(cache=None, verbose=True, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='***OPENAI_API_KEY***', openai_api_base='', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None), output_key='text', output_parser=StrOutputParser(), return_final_only=True, llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], output_parser=None, partial_variables={}, template='{page_content}', template_format='f-string', validate_template=True), document_variable_name='context', document_separator='\n\n'), input_key='query', output_key='result', return_source_documents=False, retriever=GraphSearchRetriever(tags=None, metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7fe5321bfb10>, llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['query'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Always response with comma separated list of chemical or biological terms identified in prompt.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], output_parser=None, partial_variables={}, template='{query}', template_format='f-string', validate_template=True), additional_kwargs={})]), llm=ChatOpenAI(cache=None, verbose=True, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='***OPENAI_API_KEY***', openai_api_base='', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None), output_key='text', output_parser=CommaSeparatedListOutputParser(), return_final_only=True, llm_kwargs={}), graph_search=CypherSearchAPIWrapper(graph=<llmlib.utils.lazy_neo4j_graph.LazyNeo4jGraph object at 0x7fe532106090>, related_nodes_query='\nUNWIND $terms AS term\nMATCH (s:Synonym {lowercase_name: toLower(term)})<-[:HAS_SYNONYM]-(n)\nWHERE n.eid IS NOT NULL AND NOT n.eid IN $exclude_eids\nWITH apoc.text.levenshteinSimilarity(term, n.name) as similarity, s, n, term\nORDER BY similarity DESC\nWHERE similarity > 0.5\nWITH DISTINCT term, collect({node: n, node_id: id(n), similarity: similarity})[0..$top_k_matches] as

# Test

## Initial query

In [21]:
qa.run(query, callbacks=[handler])



> Entering new RetrievalQA chain...

	>> Generating list of revelant terms
	Identified Terms:
	['Zn2+', 'glycolate']

	>> Searching for related graph nodes
	Identified Nodes:
	[{'term': 'Zn2+', 'matches': [{'node': {'original_entity_types': ['Compound'], 'eid': 'ZN+2', 'inchi': 'InChI=1S/Zn/q+2', 'abbrev_name': '', 'smiles': '[Zn+2]', 'entityType': 'Chemical', 'displayName': 'Zn2+', 'inchi_key': 'PTFCDOFLOPIGGS-UHFFFAOYSA-N', 'name': 'Zn2+', 'biocyc_id': 'ZN+2', 'data_source': 'BioCyc'}, 'similarity': 1.0, 'node_id': 3703030}]}, {'term': 'glycolate', 'matches': [{'node': {'original_entity_types': ['Compound'], 'eid': 'GLYCOLLATE', 'inchi': 'InChI=1S/C2H4O3/c3-1-2(4)5/h3H,1H2,(H,4,5)/p-1', 'abbrev_name': '', 'smiles': 'C(C(=O)[O-])O', 'entityType': 'Chemical', 'displayName': 'glycolate', 'inchi_key': 'AEMRFAOFKBGASW-UHFFFAOYSA-M', 'name': 'glycolate', 'biocyc_id': 'GLYCOLLATE', 'data_source': 'BioCyc'}, 'similarity': 1.0, 'node_id': 3703369}, {'node': {'original_entity_types': ['Chemi

'The relationship between Zn2+ and glycolate is that Zn2+ regulates the production of glycolate.'

In [22]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
system_template = """Use the following pieces of context to answer the users question. Each piece of information repsesents uncorelated cases, responses based on different pieces shoul be places into separate paragraphs.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Work on the answear in two steps:
1. Rephase pieces of context ussing natural language
2. Based on repharsed text respond to the question
----------------
{context}
----------------"""
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs=dict(
        prompt=ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_template),
            HumanMessagePromptTemplate.from_template("{question}"),
        ])
    ),
    chain_type="stuff",
    retriever=retriever,
    verbose=True,
    callbacks=[handler]
)

In [23]:
qa.run(query, callbacks=[handler])



> Entering new RetrievalQA chain...

	>> Generating list of revelant terms
	Identified Terms:
	['Zn2+', 'glycolate']

	>> Searching for related graph nodes
	Identified Nodes:
	[{'term': 'Zn2+', 'matches': [{'node': {'original_entity_types': ['Compound'], 'eid': 'ZN+2', 'inchi': 'InChI=1S/Zn/q+2', 'abbrev_name': '', 'smiles': '[Zn+2]', 'entityType': 'Chemical', 'displayName': 'Zn2+', 'inchi_key': 'PTFCDOFLOPIGGS-UHFFFAOYSA-N', 'name': 'Zn2+', 'biocyc_id': 'ZN+2', 'data_source': 'BioCyc'}, 'similarity': 1.0, 'node_id': 3703030}]}, {'term': 'glycolate', 'matches': [{'node': {'original_entity_types': ['Compound'], 'eid': 'GLYCOLLATE', 'inchi': 'InChI=1S/C2H4O3/c3-1-2(4)5/h3H,1H2,(H,4,5)/p-1', 'abbrev_name': '', 'smiles': 'C(C(=O)[O-])O', 'entityType': 'Chemical', 'displayName': 'glycolate', 'inchi_key': 'AEMRFAOFKBGASW-UHFFFAOYSA-M', 'name': 'glycolate', 'biocyc_id': 'GLYCOLLATE', 'data_source': 'BioCyc'}, 'similarity': 1.0, 'node_id': 3703369}, {'node': {'original_entity_types': ['Chemi

'Zn2+ regulates the enzyme activity of glyoxalase III, which catalyzes the reaction of H2O + glyoxal to produce H+ and glycolate. Additionally, Zn2+ also regulates the enzyme activity of glycolaldehyde dehydrogenase, which catalyzes the reaction of glycolaldehyde + NAD+ + H2O to produce glycolate + NADH + H+. Lastly, Zn2+ regulates the enzyme activity of glycolate dehydrogenase, which catalyzes the reaction of glycolate + oxidized unknown electron carrier to produce glyoxylate + reduced unknown two electron carrier.'

In [24]:
chatGPTQA(query)

<OpenAIObject chat.completion id=chatcmpl-8DsdC4EJtYTd7S4jexacdZ6lgrAjw at 0x7fe532240410> JSON: {
  "id": "chatcmpl-8DsdC4EJtYTd7S4jexacdZ6lgrAjw",
  "object": "chat.completion",
  "created": 1698318958,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The relationship between Zn2+ and glycolate is that Zn2+ can form a complex with glycolate. In this complex, the Zn2+ ion binds to the oxygen atoms in the glycolate molecule, forming coordination bonds. This interaction between Zn2+ and glycolate allows for the stabilization and transport of glycolate in biological systems."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 21,
    "completion_tokens": 77,
    "total_tokens": 98
  }
}

## Other checks

In [25]:
query1 = "What is the association between putrescine and zinT?"
qa.run(query1, callbacks=[handler])



> Entering new RetrievalQA chain...

	>> Generating list of revelant terms
	Identified Terms:
	['putrescine', 'zinT']

	>> Searching for related graph nodes
	Identified Nodes:
	[{'term': 'putrescine', 'matches': [{'node': {'original_entity_types': ['Chemical'], 'eid': '17148', 'inchi_key': 'KIDHWZJUCRJVML-UHFFFAOYSA-N', 'name': 'putrescine', 'data_source': 'ChEBI', 'alt_id': 'CHEBI:26405|CHEBI:14972|CHEBI:8650|CHEBI:45092'}, 'similarity': 1.0, 'node_id': 11208475}, {'node': {'original_entity_types': ['Compound'], 'eid': 'PUTRESCINE', 'inchi': 'InChI=1S/C4H12N2/c5-3-1-2-4-6/h1-6H2/p+2', 'abbrev_name': '', 'smiles': 'C([NH3+])CCC[NH3+]', 'entityType': 'Chemical', 'displayName': 'putrescine', 'inchi_key': 'KIDHWZJUCRJVML-UHFFFAOYSA-P', 'name': 'putrescine', 'biocyc_id': 'PUTRESCINE', 'data_source': 'BioCyc'}, 'similarity': 1.0, 'node_id': 3743644}, {'node': {'original_entity_types': ['Chemical'], 'eid': '17148', 'inchi_key': 'KIDHWZJUCRJVML-UHFFFAOYSA-N', 'name': 'putrescine', 'data_sou

'Putrescine is consumed by the reaction PUTTRANSAM-RXN, which produces L-glutamate. L-glutamate is then consumed by the reaction GLUTDECARBOX-RXN, which produces 4-aminobutanoate. ZinT is an element of the transcription unit TU8J2-2980, which is also an element of the gene BSU26830. Therefore, there is an association between putrescine and zinT through the metabolic reactions and gene regulation.'

In [26]:
chatGPTQA(query1)

<OpenAIObject chat.completion id=chatcmpl-8DsdI5uoAqEw2qoHiRe9YQpjEy3bl at 0x7fe56c3cab10> JSON: {
  "id": "chatcmpl-8DsdI5uoAqEw2qoHiRe9YQpjEy3bl",
  "object": "chat.completion",
  "created": 1698318964,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Putrescine and zinT are both associated with zinc homeostasis in bacteria.\n\nPutrescine is a polyamine compound that is involved in various cellular processes, including cell growth, differentiation, and stress response. It is synthesized from the amino acid ornithine and plays a crucial role in maintaining cellular homeostasis.\n\nZinT, on the other hand, is a zinc transporter protein found in bacteria. It is responsible for the uptake of zinc ions into the cell, which is essential for many cellular processes, including DNA replication, transcription, and enzyme activity.\n\nThe association between putrescine and zinT lies in their involvement in zinc homeostasis. Studies have shown that putrescine can bind to zinc ions and form complexes, which can then be transported into the cell by zinc transporters like ZinT. This suggests that putrescine may play a role in zinc uptake and distribution within the cell.\n\nFurthermore, the expression of zinT has been found to be regulated by putrescine levels in some bacteria. For example, in Escherichia coli, putrescine has been shown to upregulate the expression of zinT, leading to increased zinc uptake. This suggests that putrescine may act as a signaling molecule to regulate zinc homeostasis through the modulation of zinT expression.\n\nOverall, the association between putrescine and zinT lies in their involvement in zinc homeostasis, with putrescine potentially playing a role in zinc uptake and distribution, and zinT being regulated by putrescine levels."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 21,
    "completion_tokens": 329,
    "total_tokens": 350
  }
}

In [27]:
query2 = "What is the association between putrescine and zinc?"
qa.run(query2, callbacks=[handler])



> Entering new RetrievalQA chain...

	>> Generating list of revelant terms
	Identified Terms:
	['putrescine', 'zinc']

	>> Searching for related graph nodes
	Identified Nodes:
	[{'term': 'putrescine', 'matches': [{'node': {'original_entity_types': ['Chemical'], 'eid': '17148', 'inchi_key': 'KIDHWZJUCRJVML-UHFFFAOYSA-N', 'name': 'putrescine', 'data_source': 'ChEBI', 'alt_id': 'CHEBI:26405|CHEBI:14972|CHEBI:8650|CHEBI:45092'}, 'similarity': 1.0, 'node_id': 11208475}, {'node': {'original_entity_types': ['Compound'], 'eid': 'PUTRESCINE', 'inchi': 'InChI=1S/C4H12N2/c5-3-1-2-4-6/h1-6H2/p+2', 'abbrev_name': '', 'smiles': 'C([NH3+])CCC[NH3+]', 'entityType': 'Chemical', 'displayName': 'putrescine', 'inchi_key': 'KIDHWZJUCRJVML-UHFFFAOYSA-P', 'name': 'putrescine', 'biocyc_id': 'PUTRESCINE', 'data_source': 'BioCyc'}, 'similarity': 1.0, 'node_id': 3743644}, {'node': {'original_entity_types': ['Chemical'], 'eid': '17148', 'inchi_key': 'KIDHWZJUCRJVML-UHFFFAOYSA-N', 'name': 'putrescine', 'data_sou

'Putrescine is a chemical compound that is also known as CHEBI:26405, CHEBI:14972, CHEBI:8650, or CHEBI:45092. It is also referred to as putrescine in BioCyc. Its chemical structure is represented by the SMILES notation C([NH3+])CCC[NH3+]. Putrescine is also listed in MeSH with the ID D011700.\n\nZinc, on the other hand, is a chemical element. It is listed in MeSH with the ID D015032 and is also known as zinc(0) in ChEBI with the ID 30185.\n\nBased on the available information, there is no direct association mentioned between putrescine and zinc. They are two separate chemical entities with different properties.'

In [28]:
chatGPTQA(query2)

<OpenAIObject chat.completion id=chatcmpl-8DsdXrMOEE3Akx0GI62NE72dz4lKX at 0x7fe52e6572f0> JSON: {
  "id": "chatcmpl-8DsdXrMOEE3Akx0GI62NE72dz4lKX",
  "object": "chat.completion",
  "created": 1698318979,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Putrescine is a small organic compound that is naturally produced in living organisms as a result of the breakdown of amino acids. Zinc, on the other hand, is a chemical element that is essential for various biological processes in the body.\n\nThere is no direct association between putrescine and zinc. However, zinc plays a crucial role in the metabolism of amino acids, including putrescine. Zinc is required for the activity of enzymes involved in the synthesis and degradation of amino acids, including those involved in the breakdown of putrescine.\n\nAdditionally, zinc is known to have a regulatory effect on the levels of putrescine in the body. It has been observed that zinc deficiency can lead to an increase in putrescine levels, while zinc supplementation can help normalize putrescine levels.\n\nOverall, while there is no direct association between putrescine and zinc, zinc plays a significant role in the metabolism and regulation of putrescine levels in the body."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 19,
    "completion_tokens": 200,
    "total_tokens": 219
  }
}

In [29]:
query3 = "What is the association between calcium and zinT?"
qa.run(query3, callbacks=[handler])



> Entering new RetrievalQA chain...

	>> Generating list of revelant terms
	Identified Terms:
	['calcium', 'zinc']

	>> Searching for related graph nodes
	Identified Nodes:
	[{'term': 'calcium', 'matches': [{'node': {'original_entity_types': ['Chemical'], 'eid': 'D002118', 'obsolete': 0, 'name': 'Calcium', 'mesh_id': 'D002118', 'data_source': 'MeSH'}, 'similarity': 0.8571428571428571, 'node_id': 83342363}, {'node': {'original_entity_types': ['Chemical'], 'eid': '29320', 'inchi_key': 'OYPRJOBELJOOCE-UHFFFAOYSA-N', 'name': 'calcium(0)', 'data_source': 'ChEBI'}, 'similarity': 0.7, 'node_id': 11346629}, {'node': {'original_entity_types': ['Chemical'], 'eid': '22984', 'inchi_key': 'OYPRJOBELJOOCE-UHFFFAOYSA-N', 'name': 'calcium atom', 'data_source': 'ChEBI'}, 'similarity': 0.5833333333333334, 'node_id': 11332843}, {'node': {'original_entity_types': ['Chemical'], 'eid': '22984', 'inchi_key': 'OYPRJOBELJOOCE-UHFFFAOYSA-N', 'name': 'calcium atom', 'data_source': 'ChEBI'}, 'similarity': 0.583

'There is an association between calcium and zinc. Calcium is a chemical element with the symbol Ca and atomic number 20. It is also known as calcium atom. Zinc, on the other hand, is a chemical element with the symbol Zn and atomic number 30. Both calcium and zinc are essential minerals for the human body.'

In [30]:
chatGPTQA(query3)

<OpenAIObject chat.completion id=chatcmpl-8Dsdgm3F4tNioS1Nx35ZUpIEmbDV9 at 0x7fe52e6ccb90> JSON: {
  "id": "chatcmpl-8Dsdgm3F4tNioS1Nx35ZUpIEmbDV9",
  "object": "chat.completion",
  "created": 1698318988,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "There is no direct association between calcium and zinT. Calcium is a mineral that plays a crucial role in various bodily functions, including bone health, muscle contraction, nerve transmission, and blood clotting. It is primarily found in dairy products, leafy greens, and fortified foods.\n\nOn the other hand, zinT is not a recognized term or mineral. It is possible that you may be referring to zinc, which is another essential mineral involved in numerous physiological processes. Zinc is important for immune function, wound healing, DNA synthesis, and cell division. It is found in foods like meat, shellfish, legumes, and nuts.\n\nWhile both calcium and zinc are essential minerals for the body, they have distinct roles and functions and do not have a direct association with each other."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 18,
    "completion_tokens": 158,
    "total_tokens": 176
  }
}

## Check background noise

In [31]:
qa.run("What is the relationship between INHBA and MTMR4?", callbacks=[handler])



> Entering new RetrievalQA chain...

	>> Generating list of revelant terms
	Identified Terms:
	['INHBA', 'MTMR4']

	>> Searching for related graph nodes
	Identified Nodes:
	[{'term': 'INHBA', 'matches': [{'node': {'original_entity_types': ['Gene'], 'eid': '121028933', 'full_name': 'inhibin subunit beta A', 'name': 'INHBA', 'locus_tag': '', 'tax_id': '1608482', 'data_source': 'NCBI Gene'}, 'similarity': 1.0, 'node_id': 97704812}, {'node': {'original_entity_types': ['Gene'], 'eid': '119698030', 'full_name': 'inhibin subunit beta A', 'name': 'INHBA', 'locus_tag': '', 'tax_id': '1094192', 'data_source': 'NCBI Gene'}, 'similarity': 1.0, 'node_id': 97138396}, {'node': {'original_entity_types': ['Gene'], 'eid': '121333580', 'full_name': 'inhibin subunit beta A', 'name': 'INHBA', 'locus_tag': '', 'data_source': 'NCBI Gene', 'tax_id': '356909'}, 'similarity': 1.0, 'node_id': 96113143}, {'node': {'original_entity_types': ['Gene'], 'eid': '120398268', 'full_name': 'inhibin subunit beta A', 'nam

'INHBA and MTMR4 are both genes found in different organisms. INHBA is the gene that codes for the inhibin subunit beta A protein, while MTMR4 is the gene that codes for the myotubularin related protein 4. These genes have different functions and are not directly related to each other.'

## Check different qa stuffing techniques

In [32]:
from langchain.chains import RetrievalQA

In [33]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever,
    verbose=True,
    callbacks=[handler]
)
qa

RetrievalQA(memory=None, callbacks=[<langchain.callbacks.stdout.StdOutCallbackHandler object at 0x7fe530afb0d0>], callback_manager=None, verbose=True, tags=None, metadata=None, combine_documents_chain=MapReduceDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n______________________\n{context}', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]), llm=ChatOpenAI(cache=None, verbose=True, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='***OPENAI_API_KEY***', openai_api_base='', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None), output_key='text', output_parser=StrOutputParser(), return_final_only=True, llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', combine_documents_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['question', 'summaries'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['summaries'], output_parser=None, partial_variables={}, template="Given the following extracted parts of a long document and a question, create a final answer. \nIf you don't know the answer, just say that you don't know. Don't try to make up an answer.\n______________________\n{summaries}", template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]), llm=ChatOpenAI(cache=None, verbose=True, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='***OPENAI_API_KEY***', openai_api_base='', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None), output_key='text', output_parser=StrOutputParser(), return_final_only=True, llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], output_parser=None, partial_variables={}, template='{page_content}', template_format='f-string', validate_template=True), document_variable_name='summaries', document_separator='\n\n'), collapse_documents_chain=None, token_max=3000), document_variable_name='context', return_intermediate_steps=False), input_key='query', output_key='result', return_source_documents=False, retriev

In [34]:
qa.run(query, callbacks=[handler])



> Entering new RetrievalQA chain...

	>> Generating list of revelant terms
	Identified Terms:
	['Zn2+', 'glycolate']

	>> Searching for related graph nodes
	Identified Nodes:
	[{'term': 'Zn2+', 'matches': [{'node': {'original_entity_types': ['Compound'], 'eid': 'ZN+2', 'inchi': 'InChI=1S/Zn/q+2', 'abbrev_name': '', 'smiles': '[Zn+2]', 'entityType': 'Chemical', 'displayName': 'Zn2+', 'inchi_key': 'PTFCDOFLOPIGGS-UHFFFAOYSA-N', 'name': 'Zn2+', 'biocyc_id': 'ZN+2', 'data_source': 'BioCyc'}, 'similarity': 1.0, 'node_id': 3703030}]}, {'term': 'glycolate', 'matches': [{'node': {'original_entity_types': ['Compound'], 'eid': 'GLYCOLLATE', 'inchi': 'InChI=1S/C2H4O3/c3-1-2(4)5/h3H,1H2,(H,4,5)/p-1', 'abbrev_name': '', 'smiles': 'C(C(=O)[O-])O', 'entityType': 'Chemical', 'displayName': 'glycolate', 'inchi_key': 'AEMRFAOFKBGASW-UHFFFAOYSA-M', 'name': 'glycolate', 'biocyc_id': 'GLYCOLLATE', 'data_source': 'BioCyc'}, 'similarity': 1.0, 'node_id': 3703369}, {'node': {'original_entity_types': ['Chemi

'The relationship between Zn2+ and glycolate is that Zn2+ regulates the activity of enzymes involved in the production and consumption of glycolate.'

In [35]:
qa.run("What is the relationship between INHBA and MTMR4?", callbacks=[handler])



> Entering new RetrievalQA chain...

	>> Generating list of revelant terms
	Identified Terms:
	['INHBA', 'MTMR4']

	>> Searching for related graph nodes
	Identified Nodes:
	[{'term': 'INHBA', 'matches': [{'node': {'original_entity_types': ['Gene'], 'eid': '121028933', 'full_name': 'inhibin subunit beta A', 'name': 'INHBA', 'locus_tag': '', 'tax_id': '1608482', 'data_source': 'NCBI Gene'}, 'similarity': 1.0, 'node_id': 97704812}, {'node': {'original_entity_types': ['Gene'], 'eid': '119698030', 'full_name': 'inhibin subunit beta A', 'name': 'INHBA', 'locus_tag': '', 'tax_id': '1094192', 'data_source': 'NCBI Gene'}, 'similarity': 1.0, 'node_id': 97138396}, {'node': {'original_entity_types': ['Gene'], 'eid': '121333580', 'full_name': 'inhibin subunit beta A', 'name': 'INHBA', 'locus_tag': '', 'data_source': 'NCBI Gene', 'tax_id': '356909'}, 'similarity': 1.0, 'node_id': 96113143}, {'node': {'original_entity_types': ['Gene'], 'eid': '120398268', 'full_name': 'inhibin subunit beta A', 'nam

'There is no information provided in the given portion of the document about the relationship between INHBA and MTMR4.'